In [12]:
processed = []
#this script aggregates the trials into one single file that can be analyzed.

In [38]:
#move from the job bucket to the processing bucket
!gsutil mv -p gs://sahai_echo/20190814_074239/*.npy gs://torch-echo/results/

Copying gs://sahai_echo/20190814_074239/1179.npy [Content-Type=application/octet-stream]...
Removing gs://sahai_echo/20190814_074239/1179.npy...                            
Copying gs://sahai_echo/20190814_074239/1476.npy [Content-Type=application/octet-stream]...
Removing gs://sahai_echo/20190814_074239/1476.npy...                            

Operation completed over 2 objects/6.5 MiB.                                      


In [39]:
import google.datalab.storage as storage
bucket_name='torch-echo'
bucket = storage.Bucket(bucket_name)
assert bucket.exists()

results =[]

for obj in bucket.objects():
    if ".npy" in obj.key and \
        "results/" in obj.key and \
        int(obj.key[8:-4]) not in processed and \
        'QPSK' not in obj.key:
        results += [obj]

tasks = list(map(lambda r: int(r.key.split("/")[-1].split(".")[0]), results))
# tasks += list(map(lambda k: int(k.split("/")[-1].split(".")[0]), processed))
print(len(tasks))

2
False


In [14]:
missing = []

print("(", end=" ")
for i in range(max(tasks)+1):
    if i not in tasks:
        print(i, end=" ")
    missing += [i]
    
print(")", end="")

( 1179 1476 )

In [60]:
from io import BytesIO, StringIO
import numpy as np
import os
modified_tasks = list(range(1120,1230)) + list(range(1420,1530))
match = ['private_preamble-QPSK_neural_vs_classic-4_2' , 'shared_preamble-QPSK_neural_vs_clone-4_2']
tasks = modified_tasks
def write(experiments):
  #write the chunk out
  print("writing...")
  n = 0
  for identifier in experiments.keys():
    print("./%s.npy"%(identifier), os.path.isfile("./%s.npy"%(identifier)))
    trials = experiments[identifier]
    trials_npy = bucket.object("processed/%s.npy"%(identifier))
    print(trials_npy.exists())
    if os.path.isfile("./%s.npy"%(identifier)):
      saved_trials = np.load("%s.npy"%(identifier)).item()
      trials = {**saved_trials, **trials}
    elif trials_npy.exists():
      uri = trials_npy.uri
      %gcs read --object $uri --variable data
      saved_trials = np.load(BytesIO(data)).item()
      trials = {**saved_trials, **trials}
    np.save("./%s.npy"%(identifier), trials)
    n += 1
  return n

experiments = {}
current = None
to_transfer = 0 
for t in sorted(tasks):
  if to_transfer > 10:
    print("transferring...")
    !gsutil -m mv -p  ./*.npy gs://torch-echo/processed
    to_transfer = 0
  result_npy = bucket.object('results/%i.npy'%t)
  print(t, end=" ")
  processed += [t]
  uri = result_npy.uri
  %gcs read --object $uri --variable data
  result_array = np.load(BytesIO(data))
  meta, result = result_array[0], result_array[1:]
  experiment_name = meta['experiment_name']
  protocol = meta['protocol']  
  train_snr = str(meta['train_SNR_db']).replace(".","_")
  trial_num = meta['trial_num']
  identifier = "-".join([protocol, experiment_name, train_snr])
  if identifier not in match:
    continue
  print(".", end="")
  if current == None:
    current = experiment_name
  elif current != experiment_name:
    to_transfer += write(experiments)
    experiments = {}
    current = experiment_name
  trials = experiments.get(identifier, {})
  trials[trial_num] = {
    'meta' : meta,
    'data': result
  }
  experiments[identifier] = trials
  


1120 1121 1122 1123 1124 1125 1126 1127 1128 1129 1130 1131 1132 1133 1134 1135 1136 1137 1138 1139 1140 1141 1142 1143 1144 1145 1146 1147 1148 1149 1150 .1151 .1152 .1153 .1154 .1155 .1156 .1157 .1158 .1159 .1160 .1161 .1162 .1163 .1164 .1165 .1166 .1167 .1168 .1169 .1170 .1171 .1172 .1173 .1174 .1175 .1176 .1177 .1178 .1179 .1180 .1181 .1182 .1183 .1184 .1185 .1186 .1187 .1188 .1189 .1190 .1191 .1192 .1193 .1194 .1195 .1196 .1197 .1198 .1199 .1200 1201 1202 1203 1204 1205 1206 1207 1208 1209 1210 1211 1212 1213 1214 1215 1216 1217 1218 1219 1220 1221 1222 1223 1224 1225 1226 1227 1228 1229 1420 1421 1422 1423 1424 1425 1426 1427 1428 1429 1430 1431 1432 1433 1434 1435 1436 1437 1438 1439 1440 1441 1442 1443 1444 1445 1446 1447 1448 1449 1450 .writing...
./shared_preamble-QPSK_neural_vs_clone-4_2.npy False
False
1451 .1452 .1453 .1454 .1455 .1456 .1457 .1458 .1459 .1460 .1461 .1462 .1463 .1464 .1465 .1466 .1467 .1468 .1469 .1470 .1471 .1472 .1473 .1474 .1475 .1476 .1477 .1478 .1479 .

In [61]:
#last
write(experiments)
!gsutil -m mv -p  ./*.npy gs://torch-echo/processed

writing...
./private_preamble-QPSK_neural_vs_classic-4_2.npy False
False
Copying file://./private_preamble-QPSK_neural_vs_classic-4_2.npy [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file://./shared_preamble-QPSK_neural_vs_clone-4_2.npy [Content-Type=application/octet-stream]...
Removing file://./private_preamble-QPS

In [11]:
# test_npy = bucket.object('processed/gradient_passing-QPSK_classic_and_neural-13.npy')
# uri = test_npy.uri
# %gcs read --object $uri --variable data
# test = np.load(BytesIO(data)).item()
# print(test.keys(), test[32])

dict_keys([32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 31]) {'data': array([{'demod_grid_1': array([3, 3, 3, ..., 0, 0, 0]), 'batches_sent': 1, 'db_off': [4.999999999999982, 4.799999999999983, 4.399999999999984, 3.999999999999986, 3.1999999999999886, 1.5999999999999943], 'mod_std_1': [0.0, 0.0], 'test_sers': [0.00581, 0.0208, 0.0372, 0.07802, 0.14279, 0.32178], 'test_SNR_dbs': [14.599999999999913, 12.999999999999918, 11.999999999999922, 10.399999999999928, 8.399999999999935, 4.1999999999999496], 'constellation_1': array([[ 0.70710677,  0.70710677],
       [-0.70710677,  0.70710677],
       [ 0.70710677, -0.70710677],
       [-0.70710677, -0.70710677]], dtype=float32), 'test_bers': [0.002905, 0.0104, 0.0186, 0.03901, 0.07147, 0.16685]},
       {'demod_grid_1': array([3, 3, 3, ..., 0, 0, 0]), 'batches_sent': 8, 'db_off': [1.399999999999995, 1.1999999999999957, 1.1999999999999957, 0.9999999999999964, 0.7999999999999972, 0.1999999999999993], 'mod_std_1': [0.

In [49]:
os.path.isfile("./shared_preamble-QPSK_neural_vs_clone-4_2.npy")

False